# Test 4POC

In [1]:
import molsysmt as msm
import openmm as mm
from openmm import app
from openmm import unit

In [2]:
molsys = msm.convert('4POC')
molsys = msm.extract(molsys, selection='molecule_type=="protein"')
molsys = msm.build.add_missing_hydrogens(molsys)
molsys = msm.build.solvate(molsys)

#molsys = msm.convert('initial.msmpk')

In [3]:
topology = msm.convert(molsys, 'openmm.Topology')
coordinates = msm.get(molsys, coordinates=True)
positions = msm.pyunitwizard.convert(coordinates[0], to_form='openmm.unit')

In [4]:
forcefield = app.ForceField('amber14-all.xml', 'amber14/tip3p.xml')

In [5]:
system = forcefield.createSystem(topology, nonbondedMethod=app.PME,
        nonbondedCutoff=1.2*unit.nanometer, constraints=app.HBonds)

In [6]:
integrator = mm.LangevinIntegrator(300*unit.kelvin, 1.0/unit.picosecond, 2.0*unit.femtoseconds)

In [7]:
platform = mm.Platform.getPlatformByName("CPU")

In [8]:
simulation = app.Simulation(topology, system, integrator, platform)

In [9]:
simulation.context.setPositions(positions)

In [10]:
#simulation.minimizeEnergy()

#msm.convert(simulation, 'initial.msmpk')

In [11]:
simulation.context.setVelocitiesToTemperature(300*unit.kelvin)

In [12]:
from molsysmt.thirds.openmm.reporters import H5MSMReporter
from molsysmt.thirds.openmm.reporters import TQDMReporter

tqdm_reporter = TQDMReporter(10, 100)
simulation.reporters.append(tqdm_reporter)

msmh5_reporter = H5MSMReporter('traj.h5msm', 10, steps=100, selection='all',
            topology=topology, time=True, box=True, coordinates=True, velocities=False,
            potentialEnergy=True, kineticEnergy=True, temperature=True,
            includeInitialContext=True, constantReportInterval=True,
            constantStepSize=True, constantBox=True, auto_close=True)
simulation.reporters.append(msmh5_reporter)

In [13]:
simulation.step(100)

  0%|          | 0/100 [00:00<?, ?it/s]


Potential energy: -440514.74 kJ/mol ± 38312.31 kJ/mol
Temperature: 519.46 K ± 80.45 K

Execution time: 0 days, 0 hours, 0 minutes, and 15.54 seconds (1.112 ns/day).

